In [2]:
import yaml
import pandas as pd
import train # Import the modified train module

# --- Configuration ---
keep_hyperparameters = [
    'backbone',
    'batch_size',
    'bbox_loss_coef',
    # 'dropout',
    'enc_layers',
    'enc_nheads_cross',
    'eos_coef',
    'epochs',
    # 'focal_alpha',
    # 'focal_gamma',
    # 'focal_loss',
    # 'generate_dataset_runtime',
    'giou_loss_coef',
    # 'grid_size',
    'hidden_dim',
    'learning_rate',
    'learning_rate_backbone',
    'max_freq',
    'nheads',
    'num_frames',
    'num_freq_bands',
    'num_queries',
    'object_detection',
    'resize_frame',
    'scheduler_step_size',
    'self_per_cross_attn',
    'set_cost_bbox',
    'set_cost_class',
    'set_cost_giou',
    'weight_decay',
    'weight_loss_bce',
    'weight_loss_center_point',
    ]


def convert_yaml_to_csv(yaml_path):
        
    
    config = None
    descriptions = {}
    
    # 1. Get descriptions directly from the parser object
    print("Attempting to get parser from train.py...")
    try:
        parser = train._get_parser()
        for action in parser._actions:
            # Use action.dest as the key (matches the Namespace attribute)
            # Exclude the default 'help' action added by argparse
            if action.dest and action.dest != 'help':
                descriptions[action.dest] = action.help if action.help else "N/A"
        print(f"Successfully extracted {len(descriptions)} descriptions.")
    except AttributeError:
        print("Error: Could not find function 'get_parser' in train.py.")
        print("Please ensure train.py has been modified correctly.")
        exit()
    except Exception as e:
        print(f"An unexpected error occurred while getting the parser: {e}")
        exit()
    
    
    # 2. Read YAML config (same as before)
    print(f"Attempting to read configuration from: {yaml_path}")
    try:
        with open(yaml_path, 'r', encoding='utf-8') as f:
            config = yaml.load(f, Loader=yaml.FullLoader)  # Use yaml.load with FullLoader
        print("Successfully loaded YAML configuration.")
    except FileNotFoundError:
        print(f"Error: YAML file not found at '{yaml_path}'.")
        config = None
    except yaml.YAMLError as e:
        print(f"Error parsing YAML file '{yaml_path}': {e}")
        config = None
    except Exception as e:
        print(f"An unexpected error occurred while reading '{yaml_path}': {e}")
        config = None
    
    # 3. Prepare data for the table - include ALL hyperparameters from keep_hyperparameters
    table_data = []
    if descriptions:  # Only need descriptions to continue
        print("Mapping configuration values to descriptions...")
        all_keys = set(config.keys()) if config else set()
        
        # Use all keys in keep_hyperparameters regardless of whether they exist in config
        keys_to_use = keep_hyperparameters
        
        # Print hyperparameters info
        print(f"\nProcessing hyperparameters ({len(keys_to_use)} requested):")
        for key in keys_to_use:
            value = config.get(key, "None") if config else "None"
            description = descriptions.get(key, "N/A - Description not found in parser actions")
            present = "✓" if key in all_keys else "✗"
            print(f"  - {key}: {value} ({description}) [In config: {present}]")
            
            # Build table data with ALL requested hyperparameters
            table_data.append({
                "Hyperparameter": f"`{key}`",
                "Value": f"`{value}`",
                "Description": description
            })
        print(f"Table data prepared with {len(table_data)} hyperparameters.")
    else:
        print("Cannot proceed without descriptions from parser.")
    
    # 4. Create Pandas DataFrame and generate outputs
    if table_data:
        df = pd.DataFrame(table_data)
        
        # Generate and print markdown table
        markdown_table = df.to_markdown(index=False, tablefmt="github")
        print(markdown_table)
        
        # Save configuration as CSV
        csv_path = yaml_path.replace(".yaml", ".csv")
        df.to_csv(csv_path, index=False)
        print(f"\nSaved configuration to CSV: {csv_path}")
    else:
        print("\nNo data available to generate the table.")
        


convert_yaml_to_csv(
    yaml_path = "../checkpoints/output_perceiver_2025-03-31_22-10-36/config.yaml"
)

convert_yaml_to_csv(
    yaml_path = "../checkpoints/output_perceiver_detection_2025-04-23_19-42-48/config.yaml"
)

/Users/maksimploter/miniconda3/envs/sensor_dropout/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Attempting to get parser from train.py...
Successfully extracted 56 descriptions.
Attempting to read configuration from: ../checkpoints/output_perceiver_2025-03-31_22-10-36/config.yaml
Successfully loaded YAML configuration.
Mapping configuration values to descriptions...

Processing hyperparameters (26 requested):
  - backbone: cnn (Backbone type) [In config: ✓]
  - batch_size: 1 (Batch size for training) [In config: ✓]
  - bbox_loss_coef: None (L1 box coefficient) [In config: ✗]
  - enc_layers: 1 (Number of layers in Perceiver encoder) [In config: ✓]
  - enc_nheads_cross: 1 (Number of cross-attention heads) [In config: ✓]
  - eos_coef: None (Relative classification weight of the no-object class) [In config: ✗]
  - epochs: 21 (Number of training epochs) [In config: ✓]
  - giou_loss_coef: None (GIoU box coefficient) [In config: ✗]
  - hidden_dim: 128 (Latent dimension size) [In config: ✓]
  - learning_rate: 0.0001 (Learning rate) [In config: ✓]
  - learning_rate_backbone: 0.0001 (Learn

In [6]:
import os
from datetime import datetime

perceiver_parameters = [
    'enc_layers',
    'enc_nheads_cross',
    'hidden_dim',
    'max_freq',
    'nheads',
    'num_freq_bands',
    'num_queries',
    'self_per_cross_attn',
]

def csv_to_two_latex_tables(csv_path, output_dir=None, model_caption=None, training_caption=None):
    """
    Convert a CSV config file to two separate LaTeX tables:
    1. Training parameters (all except perceiver_parameters)
    2. Model parameters (only perceiver_parameters)
    
    Parameters:
    -----------
    csv_path : str
        Path to the CSV configuration file
    output_dir : str, optional
        Directory to save the output LaTeX files (defaults to CSV directory)
    model_caption : str, optional
        Custom caption for the model parameters table
    training_caption : str, optional
        Custom caption for the training parameters table
    """
    # Set default output directory if not provided
    if output_dir is None:
        output_dir = os.path.dirname(csv_path)

    # Create base filenames
    base_name = os.path.splitext(os.path.basename(csv_path))[0]

    # Read the CSV file
    try:
        df = pd.read_csv(csv_path)
        print(f"Successfully loaded CSV from: {csv_path}")
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

    # Clean up column names (remove backticks)
    df['Hyperparameter'] = df['Hyperparameter'].str.replace('`', '')
    df['Value'] = df['Value'].str.replace('`', '')

    # Split into two dataframes
    model_df = df[df['Hyperparameter'].isin(perceiver_parameters)]
    training_df = df[~df['Hyperparameter'].isin(perceiver_parameters)]

    # Generate the LaTeX tables
    model_tex_path = os.path.join(output_dir, f"{base_name}_model_params.tex")
    training_tex_path = os.path.join(output_dir, f"{base_name}_training_params.tex")

    # Set default captions if not provided
    if model_caption is None:
        model_caption = "Model architecture parameters (Perceiver)"
    if training_caption is None:
        training_caption = "Training hyperparameters"

    # Create model parameters table
    _create_latex_table(model_df, model_tex_path, model_caption, "model_params")

    # Create training parameters table
    _create_latex_table(training_df, training_tex_path, training_caption, "training_params")

    return model_tex_path, training_tex_path

def _create_latex_table(df, output_path, caption, label_suffix):
    """Helper function to create a LaTeX table from a DataFrame"""
    # Clean values for LaTeX compatibility
    df_clean = df.copy()
    for col in df_clean.columns:
        df_clean[col] = df_clean[col].apply(lambda x: _clean_for_latex(x) if isinstance(x, str) else x)
    
    # Start building the LaTeX table
    latex_content = [
        "\\begin{table}[htb!]",
        "    \\centering",
        f"    \\caption{{{caption}}}",
        f"    \\label{{tab:{label_suffix}_{datetime.now().strftime('%Y%m%d')}}}",
        "    \\begin{tblr}{width=1\\textwidth, hlines, vlines,",
        "                   colspec = { l l X },",
        "                   row{1} = {font=\\bfseries},",
        "                   colsep=3pt,",
        "                  }"
    ]
    
    # Add header row
    headers = df_clean.columns
    header_row = f"        {headers[0]} & {headers[1]} & {headers[2]} \\\\"
    latex_content.append(header_row)
    
    # Add data rows
    for _, row in df_clean.iterrows():
        row_str = f"        {row[0]} & {row[1]} & {row[2]} \\\\"
        latex_content.append(row_str)
    
    # Close the table
    latex_content.extend([
        "    \\end{tblr}",
        "\\end{table}"
    ])
    
    # Join all lines and write to file
    latex_text = "\n".join(latex_content)
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(latex_text)
    print(f"Successfully saved LaTeX table to: {output_path}")
    return output_path

def _clean_for_latex(text):
    """Clean text for LaTeX compatibility"""
    replacements = {
        '_': '\\_',
        '%': '\\%',
        '#': '\\#',
        '&': '\\&',
        '$': '\\$',
        '{': '\\{',
        '}': '\\}',
        '~': '\\textasciitilde{}',
        '^': '\\textasciicircum{}',
    }
    for char, replacement in replacements.items():
        text = text.replace(char, replacement)
    return text

model_file, training_file = csv_to_two_latex_tables(
    '../checkpoints/output_perceiver_detection_2025-04-23_19-42-48/config.csv',
    model_caption="Hyperparameters used for configuring RPerceiver model.",
    training_caption="Hyperparameters used for training the RPerceiver model for bounding box prediction task."
)

model_file, training_file = csv_to_two_latex_tables(
    '../checkpoints/output_perceiver_2025-03-31_22-10-36/config.csv',
    model_caption="Hyperparameters used for configuring RPerceiverMM model.",
    training_caption="Hyperparameters used for training the RPerceiverMM model for center point task."
)

Successfully loaded CSV from: ../checkpoints/output_perceiver_detection_2025-04-23_19-42-48/config.csv
Successfully saved LaTeX table to: ../checkpoints/output_perceiver_detection_2025-04-23_19-42-48/config_model_params.tex
Successfully saved LaTeX table to: ../checkpoints/output_perceiver_detection_2025-04-23_19-42-48/config_training_params.tex
Successfully loaded CSV from: ../checkpoints/output_perceiver_2025-03-31_22-10-36/config.csv
Successfully saved LaTeX table to: ../checkpoints/output_perceiver_2025-03-31_22-10-36/config_model_params.tex
Successfully saved LaTeX table to: ../checkpoints/output_perceiver_2025-03-31_22-10-36/config_training_params.tex


/var/folders/bv/bpq4v_wj04b281gpc675920r0000gn/T/ipykernel_67718/1984417190.py:100: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_str = f"        {row[0]} & {row[1]} & {row[2]} \\\\"
/var/folders/bv/bpq4v_wj04b281gpc675920r0000gn/T/ipykernel_67718/1984417190.py:100: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_str = f"        {row[0]} & {row[1]} & {row[2]} \\\\"
